# Linear Regression Neuron — Abalone Age Prediction
Predicting abalone **age** (Rings + 1.5) from `Length`, `Diameter`, and `Shell_weight` using a single neuron trained with gradient descent.

In [1]:
import numpy as np
import pandas as pd

## Part A: Data Setup

In [2]:
from google.colab import files
uploaded = files.upload()  # select abalone.csv

import io
csv_filename = list(uploaded.keys())[0]
df = pd.read_csv(io.BytesIO(uploaded[csv_filename]))

print('Rows:   ', len(df))
print('Columns:', df.columns.tolist())
df.head()

Saving abalone.csv to abalone.csv
Rows:    4177
Columns: ['Unnamed: 0', 'Sex', 'Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight', 'Rings']


,Unnamed: 0,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


**Inputs:** `Length`, `Diameter`, `Shell_weight`  
**Output:** `Age = Rings + 1.5` (continuous → regression problem)

In [3]:
# Target: convert Rings to Age
age = df['Rings'] + 1.5

# Select 3 input features
feature_cols = ['Length', 'Diameter', 'Shell_weight']
X = df[feature_cols].values
y = age.values

print('X shape:', X.shape)
print('y shape:', y.shape)

X shape: (4177, 3)
y shape: (4177,)


## Part B: Train-Test Split (80/20)

In [4]:
np.random.seed(42)
n_samples  = len(X)
n_train    = int(n_samples * 0.8)
shuffled_idx = np.random.permutation(n_samples)

train_idx = shuffled_idx[:n_train]
test_idx  = shuffled_idx[n_train:]

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

print('Train:', X_train.shape, '| Test:', X_test.shape)

Train: (3341, 3) | Test: (836, 3)


## Part C: Z-Score Normalisation
$z = \frac{x - \mu}{\sigma}$

> Fit on training data only to prevent data leakage. Normalisation ensures all features share the same scale so gradient descent converges efficiently.

In [5]:
train_mean = X_train.mean(axis=0)
train_std  = X_train.std(axis=0)

X_train_norm = (X_train - train_mean) / train_std
X_test_norm  = (X_test  - train_mean) / train_std

## Part D: Model — Forward Pass & Loss

In [6]:
def forward(X, weights, bias):
    """Linear prediction: y_hat = X·w + b"""
    return np.dot(X, weights) + bias


def mean_squared_error(y_true, y_pred):
    """Mean Squared Error loss."""
    return np.sum((y_true - y_pred) ** 2) / len(y_true)


def mean_absolute_error(y_true, y_pred):
    """Mean Absolute Error metric."""
    return np.sum(np.abs(y_true - y_pred)) / len(y_true)

## Part E: Gradient Descent
Gradient = direction & magnitude of loss change w.r.t. each parameter.  
**Subtracting** the gradient moves weights in the direction that *reduces* loss.

In [7]:
def compute_grad_weights(X, y_true, y_pred):
    """Gradient of MSE w.r.t. weights."""
    return (-2 / len(y_true)) * np.dot(X.T, (y_true - y_pred))


def compute_grad_bias(y_true, y_pred):
    """Gradient of MSE w.r.t. bias."""
    return (-2 / len(y_true)) * np.sum(y_true - y_pred)

## Part F: Training Loop

In [8]:
weights       = np.random.uniform(low=-1, high=1, size=X_train_norm.shape[1])
bias          = 0.0
learning_rate = 0.01
num_epochs    = 1000
loss_history  = []

for epoch in range(num_epochs):
    y_pred       = forward(X_train_norm, weights, bias)
    current_loss = mean_squared_error(y_train, y_pred)
    grad_w       = compute_grad_weights(X_train_norm, y_train, y_pred)
    grad_b       = compute_grad_bias(y_train, y_pred)

    weights -= learning_rate * grad_w
    bias    -= learning_rate * grad_b
    loss_history.append(current_loss)

    if epoch % 100 == 0:
        print(f'Epoch {epoch:>4}  |  Loss: {current_loss:.4f}')

Epoch    0  |  Loss: 146.2963
Epoch  100  |  Loss: 9.0235
Epoch  200  |  Loss: 6.6691
Epoch  300  |  Loss: 6.5700
Epoch  400  |  Loss: 6.5321
Epoch  500  |  Loss: 6.5081
Epoch  600  |  Loss: 6.4920
Epoch  700  |  Loss: 6.4808
Epoch  800  |  Loss: 6.4725
Epoch  900  |  Loss: 6.4660


## Part G: Evaluation

In [9]:
y_test_pred = forward(X_test_norm, weights, bias)

test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f'Test MSE: {test_mse:.4f}')
print(f'Test MAE: {test_mae:.4f}')

Test MSE: 5.6718
Test MAE: 1.7762


In [10]:
# Compare true vs predicted age for first 5 test samples
true_ages      = y_test          # already Age = Rings + 1.5
predicted_ages = y_test_pred
abs_errors     = np.abs(true_ages - predicted_ages)

results_df = pd.DataFrame({
    'True Age':      true_ages[:10].round(2),
    'Predicted Age': predicted_ages[:10].round(2),
    'Abs Error':     abs_errors[:10].round(2)
})
print(results_df.to_string(index=False))

 True Age  Predicted Age  Abs Error
     10.5           9.63       0.87
     11.5          13.07       1.57
     10.5          11.74       1.24
     11.5           9.20       2.30
      7.5           8.60       1.10
      9.5          10.31       0.81
      9.5          10.33       0.83
      8.5           9.11       0.61
     11.5          12.35       0.85
     10.5          14.37       3.87
